# **1. Tratamento de dados para nosso setup**

Nessa etapa, basicamente estamos preparando nossos dados, assim como feito no EDA. Dessa maneira, evitamos potenciais erros na nossa modelagem.

In [47]:
import pandas as pd
import numpy as np

df = pd.read_csv('desafio_indicium_imdb.csv')

# Linha de código paliativo, evitando a criação de um índice antigo.
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)

# Corrigindo 'Runtime' e 'Gross'.
df['Runtime'] = df['Runtime'].str.replace(' min', '').astype(int)

df['Gross'] = df['Gross'].str.replace(',', '', regex=False)
df['Gross'] = pd.to_numeric(df['Gross'], errors='coerce')

# Corrigindo 'Released_Year'.
df['Released_Year'] = pd.to_numeric(df['Released_Year'], errors='coerce')
df.loc[df['Series_Title'] == 'Apollo 13', 'Released_Year'] = 1995

# Imputação.
median_year = df['Released_Year'].median()
df['Released_Year'] = df['Released_Year'].fillna(median_year)

df['Certificate'] = df['Certificate'].fillna('Not Rated')

median_meta_score = df['Meta_score'].median()
df['Meta_score'] = df['Meta_score'].fillna(median_meta_score)

median_gross = df['Gross'].median()
df['Gross'] = df['Gross'].fillna(median_gross)

# Garantindo que o ano seja um número inteiro.
df['Released_Year'] = df['Released_Year'].astype(int)
#Verificando se a preparação de dados foi bem-sucedida.
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series_Title   999 non-null    object 
 1   Released_Year  999 non-null    int64  
 2   Certificate    999 non-null    object 
 3   Runtime        999 non-null    int64  
 4   Genre          999 non-null    object 
 5   IMDB_Rating    999 non-null    float64
 6   Overview       999 non-null    object 
 7   Meta_score     999 non-null    float64
 8   Director       999 non-null    object 
 9   Star1          999 non-null    object 
 10  Star2          999 non-null    object 
 11  Star3          999 non-null    object 
 12  Star4          999 non-null    object 
 13  No_of_Votes    999 non-null    int64  
 14  Gross          999 non-null    float64
dtypes: float64(3), int64(3), object(9)
memory usage: 117.2+ KB


# **2. Preparação de Dados para Modelagem**

Nessa etapa, nós vamos separar nossos dados em dois grupos :     

- Variáveis Preditoras (Features) : O `x`, são características do filme que acreditamos que contêm informações para prever a nota. Na minha modelagem, escolhi `Released_Year`, `Runtime`, `Meta_score`, `No_of_Votes` e `Gross`.

- Variável Alvo (Target) : O `y`, é o que queremos que nosso modelo aprenda a prever. Nesse caso, `IMDB_Rating`.

Notar ainda que resolvi separar meu **sample** em dois, com 80% sendo voltados para amostras de **treino**, permitindo que nosso modelo possa aprender os padrões de forma robusta e suficiente, e 20% sendo voltados para amostras de **teste**, de forma a testar a performance final do modelo.

Por fim, devemos observar que as variáveis categóricas contam com centenas de valores únicos, o que tornaria nosso problema de modelagem bastante complexo dada sua grande cardinalidade, desse modo, resolvi manter apenas as 50 categorias mais frequentes (por exemplo, os 50 diretores mais comuns), agrupando o restante em uma única categoria chamada de "Other".








In [48]:
from sklearn.model_selection import train_test_split

# 1. Selecionando Features e Alvo
features = ['Released_Year', 'Runtime', 'Meta_score', 'No_of_Votes', 'Gross', 'Director', 'Star1', 'Genre']
target = 'IMDB_Rating'

X = df[features].copy()
y = df[target]

# 2. Limitando a cardinalidade para evitar excesso de colunas.
for col in ['Director', 'Star1', 'Genre']:
    top_categories = X[col].value_counts().nlargest(50).index
    X.loc[:, col] = X[col].where(X[col].isin(top_categories), 'Other')

# 3. Divisão em Dados de Treino e de Teste
# Separação de 20% dos dados para teste, para avaliar o modelo final de forma imparcial.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dados de treino: {X_train.shape[0]} amostras")
print(f"Dados de teste: {X_test.shape[0]} amostras")


Dados de treino: 799 amostras
Dados de teste: 200 amostras


# **3. Criação de um Pipeline de Pré-Processamento.**

Nessa etapa, utilizarei de um `Pipeline` para cuidar do pré-processamento, garantindo desse modo que minhas transformações ocorram de forma automática, além de evitar problemas de vazamento de dados.

Como citado anteriormente, as variáveis categóricas, `Director`, `Genre` e `Star1`, possuem centenas de valores únicos, sendo dados de texto, cujos modelos não conseguem processar diretamente. Desse modo, foi necessário utilizar de uma técnica de **One-Hot Encoding**, basicamente transformando essas 3 categorias em vetores numéricos binários, com cada categoria tornando-se uma nova coluna com valores de 0 ou 1.

In [49]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Definindo colunas numéricas e categóricas
categorical_features = ['Director', 'Star1', 'Genre']
numeric_features = ['Released_Year', 'Runtime', 'Meta_score', 'No_of_Votes', 'Gross']

# Aplicando o OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features) # sparse_output=False para facilitar a integração com alguns modelos
    ],
    remainder='passthrough'
)



# **4. Baseline**

É nessa etapa que podemos responder mais uma das questões do Desafio :     

**Qual tipo de problema estamos resolvendo (Regressão ou Classificação)?**

Como já deve ter ficado claro, trata-se de um problema de **regressão** devido a natureza da nossa variável alvo `y`, como sendo um valor numérico contínuo, nesse caso, o `IMDB_Rating`.

É nesse sentido que nessa etapa utilizarei uma abordagem em que testarei vários **algortimos de regressão**, comparando suas performances usando uma técnica de **Cross-Validation**. Como métrica, utilizei o **Erro Médio Absoluto (MAE)**, por ser mais simples de interpretar.




In [50]:
# Baseline de modelos de regressão
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

# Lista de modelos para teste
models = {
    'Linear Regression': LinearRegression(),
    'K-Nearest Neighbors': KNeighborsRegressor(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}

results = {}

# Loop para treino e avaliação de cada modelo
for name, model in models.items():
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    # Etapa de Cross-Validation
    scores = cross_val_score(model_pipeline, X_train, y_train, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)

    # Armazenamos o MAE médio (notar que vamos inverter o sinal)
    results[name] = -np.mean(scores)
    print(f"Modelo: {name} | MAE Médio (CV): {-np.mean(scores):.4f}")

# Exibição do melhor modelo
best_model_name = min(results, key=results.get)
print(f"\n✅ Melhor modelo no baseline: {best_model_name} com MAE de {results[best_model_name]:.4f}")

Modelo: Linear Regression | MAE Médio (CV): 0.1716
Modelo: K-Nearest Neighbors | MAE Médio (CV): 0.2304
Modelo: Decision Tree | MAE Médio (CV): 0.1931
Modelo: Random Forest | MAE Médio (CV): 0.1507
Modelo: Gradient Boosting | MAE Médio (CV): 0.1483

✅ Melhor modelo no baseline: Gradient Boosting com MAE de 0.1483


Com o resultado em mãos, notamos que **Random Forest** e **Gradient Boosting**, que são modelos de **ensemble** apresentam os dois menores valores de MAE Médio, indicando desempenho muito superior em relação aos modelos mais simples.

Afim de resolução do desafio, optei pela escolha do **Random Forest**, apesar do Gradient Boosting ter apresentado o menor MAE médio absoluto, isso porque ele possui uma implementação relativamente mais simples em comparação com o Gradient Boosting.

# **5. Treinamento e Avaliação do Modelo Final**

Nessa etapa, vamos treinar e avaliar o modelo vencedor, no caso, o **Random Forest**.

In [51]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Criação do pipeline final com o modelo que escolhemos a partir do baseline.
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))
])

# Treinamento com o conjunto de treino.
model.fit(X_train, y_train)

# Avaliação final no conjunto de teste, que o modelo nunca viu.
y_pred_test = model.predict(X_test)

# Cálculo das métricas finais
mae = mean_absolute_error(y_test, y_pred_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
r2 = r2_score(y_test, y_pred_test)


print(f"Erro Médio Absoluto (MAE): {mae:.4f}")
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse:.4f}")
print(f"Coeficiente de Determinação (R²): {r2:.4f}")



Erro Médio Absoluto (MAE): 0.1541
Raiz do Erro Quadrático Médio (RMSE): 0.1986
Coeficiente de Determinação (R²): 0.3990


# **6. Teste do Modelo escolhido**

Por fim, vamos resolver a questão do Desafio em que devemos prever a nota IMDB do filme dado, utilizando o nosso modelo escolhido e treinado.

In [52]:


# Dados do filme
new_movie_data = {
    'Released_Year': 1994,
    'Runtime': 142,
    'Meta_score': 80.0,
    'No_of_Votes': 2343110,
    'Gross': 28341469.0,
    'Director': 'Frank Darabont',
    'Star1': 'Tim Robbins',
    'Genre': 'Drama'
}
new_movie_df = pd.DataFrame([new_movie_data])

# Aplicação da limitação de cardinalidade
for col in ['Director', 'Star1', 'Genre']:
    top_categories = X[col].value_counts().nlargest(50).index
    if new_movie_df[col].iloc[0] not in top_categories:
        new_movie_df.loc[0, col] = 'Other'

# Fazendo a previsão com o modelo treinado na célula anterior
predicted_rating = model.predict(new_movie_df[features])
final_score = predicted_rating[0]

print(f"A nota do IMDB prevista para o filme 'The Shawshank Redemption' é: {final_score:.2f}")

A nota do IMDB prevista para o filme 'The Shawshank Redemption' é: 8.77


# **7. Salvando o modelo em formato .pkl**

In [53]:
import pickle

filename = 'imdb_rating_predictor.pkl'

with open(filename, 'wb') as f:
    pickle.dump(model, f)

